In [1]:
# Librerias necesarias
import os
import sys
import json
import random
import numpy as np
from collections import deque
# Add the parent directory to sys.path to allow relative imports
sys.path.append(os.path.abspath(os.path.join('..', '1. codigo analisis')))
import parametros as p

In [2]:
# Cargo los datos necesarios y transformo las llaves nuevamente a int (que se habian vuelto str)
incertidumbre = {}
for hospital in range(0,4): # 0 son llegadas a WL
    incertidumbre[hospital] = {}
    for requerimiento in range(1,4):
        incertidumbre[hospital][requerimiento] = {}

with open("resultados incertidumbre/incertidumbre_simulada.json", "r") as file:
    incertidumbre_keys_str = json.load(file)

for hospital in range(0,4): # 0 son llegadas a WL
    for requerimiento in range(1,4):
        for grd in range(1,9):
            incertidumbre[hospital][requerimiento][grd] = []
            lista = incertidumbre_keys_str[str(hospital)][str(requerimiento)][str(grd)]
            if lista != []:
                for data in lista:
                    arreglado = {
                    'TI': data["TI"],
                    'camino': {1: data['camino']["1"], 2: data['camino']["2"], 3: data['camino']["3"]},
                    'espera': {1: data['espera']["1"], 2: data['espera']["2"], 3: data['espera']["3"]}
                    }
                    incertidumbre[hospital][requerimiento][grd].append(arreglado)
            else:
                incertidumbre[hospital][requerimiento][grd] = []


In [3]:
class Paciente:
    CONTADOR_ID = 1  # Variable de clase para contar el número de Pacientes creados

    def __init__(self, hospital_llegada, requerimiento_inicial, grd, datos_incertidumbre):
        # Atributos del paciente al llegar, no cambian
        self.id = Paciente.CONTADOR_ID  # Asigna un ID único al Paciente
        self.hospital_llegada = hospital_llegada
        self.requerimiento_inicial = requerimiento_inicial
        self.grd = grd
        self.ti_inicial = datos_incertidumbre["TI"]
        self.camino = datos_incertidumbre["camino"]
        self.espera = datos_incertidumbre["espera"]

        # Diccionarios parametros unidades y hospitales inverso (para simplificar el acceso)
        self.dict_hospitales = self.invertir_dict(p.dict_hospitales)
        self.dict_unidades = self.invertir_dict(p.dict_unidades)

        # Atributos que cambian a lo largo de la simulación
        self.log_eventos = []
        self.ti_evento_actual = datos_incertidumbre["TI"]
        self.tiempo_actual = datos_incertidumbre["TI"]
        self.hospital_actual = hospital_llegada
        self.unidad_actual = self.unidad_llegada()
        self.unidad_requerida = self.requerimiento_inicial
        self.esperando = True # True si esta esperando y no siendo tratado (revisar en cada ciclo)
        self.costo_social = 0
        self.costos_operativos = 0
        Paciente.CONTADOR_ID += 1  # Incrementa el contador de Pacientes

    # Necesarias para cuando se crean los pacientes
    def invertir_dict(self, diccionario):
        invertido = {v: k for k, v in diccionario.items()}
        return invertido
    
    def unidad_llegada(self):
        if self.hospital_llegada == 0:
            unidad_actual = p.dict_unidades["WL"]
        else:
            unidad_actual = p.dict_unidades["ED"]
        return unidad_actual
    
    # Necesarias durante la simulación

    # Guarda los eventos que le ocurren al paciente en el log_eventos
    def agregar_log_evento(self):
        if self.hospital_actual == 0 and self.unidad_actual == p.dict_unidades["WL"]:
            hospital = "WL"
        elif self.unidad_actual == p.dict_unidades["PS"]:
            hospital = "PS"
        else:
            hospital = self.dict_hospitales[self.hospital_actual]
        new_row = {
            'ID': self.id,
            'MS_GRD': self.grd,
            'UBICACIÓN': hospital + "_" + self.dict_unidades[self.unidad_actual],
            'TI': self.ti_evento_actual,
            'TF': self.tiempo_actual,
            'HOSPITAL': hospital,
            'UNIDAD': self.dict_unidades[self.unidad_actual]
        }
        self.log_eventos.append(new_row)
    
    # Metodos para calcular los distintos costos en los que incurre el paciente (social y operativo)
    def costo_espera(self, tiempo_espera = 1): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_espera_wl[self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["GA"]: 
            costo = p.dict_costo_espera_ga[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["ED"]: 
            costo = p.dict_costo_espera_ed[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual in (p.dict_unidades["OR"], p.dict_unidades["ICU"], p.dict_unidades["SDU/WARD"]): # (1, 2, 3)
            costo = p.dict_costo_espera_hospitalizado[self.hospital_actual][self.grd][self.unidad_actual][self.unidad_requerida] * tiempo_espera
        else:
            costo = 0
        return costo

    def costo_traslado(self, hospital_destino): # Revisado, funciona bien
        costo = p.dict_costo_traslado[self.hospital_actual][hospital_destino][self.grd][self.requerimiento_inicial]
        return costo
    
    def costo_desvio(self): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["ED"]:
            costo = p.dict_costo_derivar_ed[self.hospital_actual][self.grd][self.requerimiento_inicial]
        elif self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_derivar_wl[self.grd][self.requerimiento_inicial]
        return costo

    # Metodo para cambio de unidad o PS, creo el log correspondiente antes de cambiar la unidad
    def cambiar_unidad(self, hospital: int, id_unidad: int):
        # Agrego el evento al log
        self.agregar_log_evento() # Esto toma como TF el tiempo actual que es el TI de la unidad destino
        # Actualizo el tiempo de evento actual, osea el TI en la nueva unidad
        self.ti_evento_actual = self.tiempo_actual
        # Actualizo el hospital actual
        self.hospital_actual = hospital
        # Actualizo la unidad actual
        self.unidad_actual = id_unidad
        # Corroboro si sigue esperando o esta siendo tratado
        self.espera_o_tratamiento()
        
        # Casos de termino de la simulacion
        if self.unidad_actual == p.dict_unidades["PS"]:
            self.agregar_log_evento()

    def actualizar_tiempo(self):
        self.tiempo_actual += 1
        print("actualizando tiempo")
        pass
    
    # Ve estado actual, requerimiento y modifica esperando acordemente, tambien modifica unidad_requerida
    def espera_o_tratamiento(self):
        pass



    def __str__(self):
        return f"Paciente {self.id} | GRD: {self.grd} | Hospital actual: {self.hospital_actual} | Unidad: {self.unidad_actual} | Requerimiento: {self.unidad_requerida} | Tiempo actual: {self.tiempo_actual}"


In [4]:
# Crear un diccionario con un paciente de cada GRD y requerimiento (para WL)
d_paci = {}
hospital = 1
for grd in range(1, 9):  # GRD values
    d_paci[grd] = {}
    for requerimiento in range(1, 4):  # Requerimiento values
        d_paci[grd][requerimiento] = []
        for cantidad in range(random.randint(1, 5)):  # Cantidad de pacientes por combinación
            # Crear un paciente con datos ficticios para cada combinación
            datos_incertidumbre = {
                "TI": 0,
                "camino": {1: [], 2: [], 3: []},
                "espera": {1: 0, 2: 0, 3: 0}
            }
            paciente = Paciente(hospital_llegada=hospital, requerimiento_inicial=requerimiento, grd=grd, datos_incertidumbre=datos_incertidumbre)
            # paciente.unidad_actual = p.dict_unidades["OR"]
            d_paci[grd][requerimiento].append(paciente)

# Se instancian a todos los pacientes a partir de los datos de incertidumbre
pacientes = {}
lista_pacientes = []
for hospital in range(0,4): # 0 son llegadas a WL
    pacientes[hospital] = {}
    for requerimiento in range(1,4):
        pacientes[hospital][requerimiento] = {}
        for grd in range(1,9):
            pacientes[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for i in range(cantidad_pacientes):
                    paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                    pacientes[hospital][requerimiento][grd].append(paciente)
                    lista_pacientes.append(paciente)
            else:
                pacientes[hospital][requerimiento][grd] = []

# Se generan tantas listas como pacientes haya
def agrupar_pacientes_por_llegada(pacientes):
    pacientes_separados_por_llegada = {}
    for paciente in pacientes:
        dia = paciente.ti_inicial
        if dia not in pacientes_separados_por_llegada:
            pacientes_separados_por_llegada[dia] = []
        pacientes_separados_por_llegada[dia].append(paciente)
    return pacientes_separados_por_llegada

pacientes_separados_por_llegada = agrupar_pacientes_por_llegada(lista_pacientes)

# Aqui podemos ver los objetos de clase Pacientes que llegaron en x ciclo
# x = 12
# pacientes_separados_por_llegada.get(x)

In [5]:
class WL:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    al usar deques las operaciones de append y popleft son O(1), lo cual lo hace mas eficiente
    al ser un sistema de colas FIFO"""
    def __init__(self, requerimientos: list, grds: list):
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
    
    # Siempre el primero de cada sublista es el que lleva mas esperando porque llego antes
    def sacar_paciente(self, requerimiento: int, grd: int): # Revisado, funciona bien
        if self.sub_listas[requerimiento][grd]:
            return self.sub_listas[requerimiento][grd].popleft()
        else:
            return None
    
    def acumular_costo_ciclo(self): # Revisado, funciona bien
        for requerimiento in self.sub_listas:
            for grd in self.sub_listas[requerimiento]:
                if self.sub_listas[requerimiento][grd]:
                    for paciente in self.sub_listas[requerimiento][grd]:
                        self.costo_acumulado_sub_listas[requerimiento][grd] += paciente.costo_espera()
                else:
                    self.costo_acumulado_sub_listas[requerimiento][grd] += 0

    def __str__(self):
        return self.sub_listas, self.costo_acumulado_sub_listas

In [6]:
class PS:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    funciona simplemente como un sumidero de pacientes"""
    def __init__(self, requerimientos: list, grds: list):
        self.id_unidad = p.dict_unidades["PS"]
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Igual que WL
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente):
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        self.costo_acumulado_sub_listas[paciente.requerimiento_inicial][paciente.grd] += paciente.costo_desvio() # Revisado, funciona bien
        paciente.cambiar_unidad(paciente.hospital_actual, self.id_unidad)

    def __str__(self):
        print("Sublistas de PS:")
        display(self.sub_listas)
        print("Costos acumulados de PS:")
        display(self.costo_acumulado_sub_listas)
        pass

In [7]:
class Termino:
    """
    Funciona con varias sublistas, una por cada combinación de requerimiento y grd,
    almacena pacientes terminados.
    """
    def __init__(self, requerimientos: list, grds: list):
        self.id_unidad = p.dict_unidades["END"]
        self.sub_listas = {}
        self.crear_sublistas(requerimientos, grds)

    def crear_sublistas(self, requerimientos: list, grds: list):
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()

    def agregar_paciente(self, paciente: Paciente):
        """
        Agrega un paciente a la sublista correspondiente según su requerimiento_inicial y grd.
        """
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        paciente.cambiar_unidad(paciente.hospital_actual, self.id_unidad)

    def __str__(self):
        lines = ["Sublistas de Termino:"]
        for req, dict_grd in self.sub_listas.items():
            for grd, dq in dict_grd.items():
                lines.append(f"  - Requerimiento {req} GRD {grd}: {len(dq)} pacientes")
        return "\n".join(lines)

In [35]:
"""Todas las unidades de los hospitales heredan de la clase Unidad, se comportan exactamente igual
solo varia el id de la unidad y la capacidad, que se obtiene del diccionario de capacidades"""
class Unidad:
    """Clase para las unidades, se usan listas para almacenar pacientes debido a su pequeño tamaño, 
    en el peor de los casos el SDU/WARD tiene 165 pacientes, por lo que una lista es suficiente y
    es facil de iterar sobre ella"""
    def __init__(self, id_unidad, hospital, capacidad):
        self.id_unidad = id_unidad
        self.hospital = hospital
        self.capacidad = capacidad
        self.pacientes = []

    @property # Revisado, funciona bien
    def ocupacion(self):
        return len(self.pacientes)
    
    @property # Revisado, funciona bien
    def ocupacion_porcentual(self):
        return len(self.pacientes)/self.capacidad
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        if self.ocupacion < self.capacidad and paciente not in self.pacientes:
            # Si el paciente ya esta en la lista, no se agrega
            # Si la ocupacion es menor a la capacidad, se agrega el paciente
            self.pacientes.append(paciente)
            # Al agregar el paciente, se actualiza su unidad actual con el metodo de la clase
            paciente.cambiar_unidad(self.hospital, self.id_unidad)
            return True
        else:
            return False
        
    def sacar_paciente(self, paciente): # Revisado, funciona bien
        if self.ocupacion > 0 and paciente in self.pacientes:
            # Si el paciente no esta en la lista, no se saca
            # Si la ocupacion es mayor a 0, se saca el paciente
            # Cada instancia de paciente es unica, por lo que no hay problema al usar remove
            self.pacientes.remove(paciente) 
            return paciente
        else:
            return False
    
    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada paciente de la unidad
        for paciente in self.pacientes:
            paciente.actualizar_tiempo()
    
    def __str__(self):
        # Diccionario invertido para obtener el nombre de la unidad
        unit_names = {
            p.dict_unidades["OR"]: "OR",
            p.dict_unidades["ICU"]: "ICU",
            p.dict_unidades["SDU/WARD"]: "SDU/WARD",
            p.dict_unidades["GA"]: "GA",
            p.dict_unidades["ED"]: "ED"
        }
        unit_name = unit_names.get(self.id_unidad, f"Unidad {self.id_unidad}")
        description = f"{unit_name} (Hospital {self.hospital}): {self.ocupacion}/{self.capacidad} pacientes"
        if self.pacientes:
            description += f"\nPacientes: {len(self.pacientes)}, {self.ocupacion_porcentual:.2%} ocupación"
            if len(self.pacientes) <= 20:  # Only show details for a small number of patients
                for i, paciente in enumerate(self.pacientes):
                    description += f"\n  {i+1}. Paciente ID {paciente.id}, GRD {paciente.grd}"
        return description

class Or(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["OR"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Icu(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["ICU"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class SduWard(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["SDU/WARD"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Ga(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["GA"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Ed(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["ED"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

In [36]:
# Testeo x para ver si funcionan las unidades, si funcionaron
# Se instancian a todos los pacientes a partir de los datos de incertidumbre
pacientes = {}
lista_pacientes = []
for hospital in range(0,4): # 0 son llegadas a WL
    pacientes[hospital] = {}
    for requerimiento in range(1,4):
        pacientes[hospital][requerimiento] = {}
        for grd in range(1,9):
            pacientes[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for i in range(cantidad_pacientes):
                    paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                    pacientes[hospital][requerimiento][grd].append(paciente)
                    lista_pacientes.append(paciente)
            else:
                pacientes[hospital][requerimiento][grd] = []

# Se generan tantas listas como pacientes haya
def agrupar_pacientes_por_llegada(pacientes):
    pacientes_separados_por_llegada = {}
    for paciente in pacientes:
        dia = paciente.ti_inicial
        if dia not in pacientes_separados_por_llegada:
            pacientes_separados_por_llegada[dia] = []
        pacientes_separados_por_llegada[dia].append(paciente)
    return pacientes_separados_por_llegada

pacientes_separados_por_llegada = agrupar_pacientes_por_llegada(lista_pacientes)

pacientes_prueba = pacientes_separados_por_llegada.get(2)

# Prueba unidades
or_1 = Or(1)
ed_1 = Ed(1)
icu_1 = Icu(1)
sdu_ward_1 = SduWard(1)
ga_1 = Ga(1)
termino = Termino([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
print(or_1)
for paciente in pacientes_prueba[0:4]:
    or_1.agregar_paciente(paciente)
print(or_1)

# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# or_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# print(or_1.sacar_paciente(pacientes_prueba[0]))
# print(or_1.sacar_paciente(pacientes_prueba[0]))
# print(or_1.sacar_paciente(pacientes_prueba[0]))
# print(or_1.agregar_paciente(pacientes_prueba[0]))
# print(or_1.agregar_paciente(pacientes_prueba[0]))
# ed_1.agregar_paciente(paciente_sacado)
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# icu_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# ga_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# sdu_ward_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# termino.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)

   

OR (Hospital 1): 0/6 pacientes
OR (Hospital 1): 4/6 pacientes
Pacientes: 4, 66.67% ocupación
  1. Paciente ID 3662677, GRD 6
  2. Paciente ID 3669517, GRD 7
  3. Paciente ID 3669518, GRD 7
  4. Paciente ID 3669519, GRD 7


In [ ]:
class Hospital:
    """Representa un hospital con una instancia de cada unidad disponible."""
    def __init__(self, hospital_id):
        self.hospital = hospital_id
        # Instanciar una unidad de cada tipo
        self.OR = Or(hospital_id)
        self.ICU = Icu(hospital_id)
        self.SDU_WARD = SduWard(hospital_id)
        self.GA = Ga(hospital_id)
        self.ED = Ed(hospital_id)
        # Diccionario para acceso y recorridos
        self.unidades = {
            p.dict_unidades["OR"]: self.OR,
            p.dict_unidades["ICU"]: self.ICU,
            p.dict_unidades["SDU/WARD"]: self.SDU_WARD,
            p.dict_unidades["GA"]: self.GA,
            p.dict_unidades["ED"]: self.ED
        }

    def actualizar_tiempo(self):
        # Llama al método actualizar_tiempo de cada unidad del hospital
        for unidad in self.unidades.values():
            unidad.actualizar_tiempo()
    
    def ocupacion(self):
        # Devuelve la ocupación de todo el hospital
        ocupacion = {
            unidad_id: (unidad.ocupacion_porcentual, unidad.ocupacion) for unidad_id, unidad in self.unidades.items()
        }
        return ocupacion
    
    def agregar_paciente(self, paciente: Paciente, unidad: int):
        # Agrega un paciente a la unidad correspondiente.
        if unidad in self.unidades:
            return self.unidades[unidad].agregar_paciente(paciente)
        else:
            raise ValueError(f"Unidad {unidad} no válida en el hospital {self.hospital}.")
    
    def sacar_paciente(self, paciente: Paciente, unidad: int):
        # Saca un paciente de la unidad correspondiente.
        if unidad in self.unidades:
            return self.unidades[unidad].sacar_paciente(paciente)
        else:
            raise ValueError(f"Unidad {unidad} no válida en el hospital {self.hospital}.")

    def __str__(self):
        lines = [f"Hospital {self.hospital}:"]
        for nombre, unidad in self.unidades.items():
            lines.append(f"  - {nombre}: {unidad.ocupacion}/{unidad.capacidad_maxima}")
        return "\n".join(lines)

In [12]:
class Simulacion:

    def __init__(self):
        pass

    def __str__(self):
        pass